#### Notebook example to run EMR serverless job from Sagemaker

us-east-1 applications: <br>
* pd-autoencoder-ad-v1 : 00f64bef5869kl09
* pd-autoencoder-ad-v2 : 00f66ohicnjchu09
* pd-autoencoder-ad-v3 : 00f6959e87i48609
<br>

us-west-2 applications: <br>
* pd-autoencoder-ad-container-v1


Note: while launching your job, please make sure of the region from where you are running it.
jobs for us-east-region can only be launched from us-east-1 

In [2]:
from emr_serverless import EMRServerless

In [3]:
# application id
application_id = '00f66mmuts7enm09'
# don't chnage the serverless_job_role_arn
serverless_job_role_arn = 'arn:aws:iam::064047601590:role/hamza-emr-serverless-role'
# s3 bukcet name where the dependencies, logs and code sits
s3_bucket_name = 'emr-serverless-output-pd'
emr_emtry_point = 's3://emr-serverless-output-pd/code/pyspark/pd-autoencoder-ad/s3_test_emr.py'
zipped_env_path = 's3://emr-serverless-output-pd/code/pyspark/pd-autoencoder-ad/pyspark_deps_all_rec_types_v2.tar.gz'

In [4]:
emr_serverless = EMRServerless(application_id)

In [5]:
print("Starting EMR Serverless Spark App")
#emr_serverless.create_application("pd-autoencoder-test-emr-cli", "emr-6.6.0")
emr_serverless.start_application(application_id)
print(emr_serverless)

Starting EMR Serverless Spark App
EMR Serverless SPARK Application: 00f66mmuts7enm09


In [6]:
# Run (and wait for) a Spark job
print("Submitting new Spark job")
job_run_id = emr_serverless.run_spark_job(
    script_location=emr_emtry_point,
    job_role_arn=serverless_job_role_arn,
    application_id = application_id,
    arguments=[f"s3://{s3_bucket_name}/emr-serverless/output"],
    s3_bucket_name=s3_bucket_name,
    zipped_env_path = zipped_env_path
)

Submitting new Spark job


In [7]:
job_status = emr_serverless.get_job_run(job_run_id)
print(f"Job finished: {job_run_id}, status is: {job_status.get('state')}")

# Fetch and print the logs
spark_driver_logs = emr_serverless.fetch_driver_log(s3_bucket_name, job_run_id)
print("File output from stdout.gz:\n----\n", spark_driver_logs, "\n----")

# Now stop and delete your application
#print("Stopping App")
#emr_serverless.stop_application()
#emr_serverless.delete_application()
print("Done! 👋")

Job finished: 00f697tg11ptuk09, status is: SUCCESS
File output from stdout.gz:
----
 2022-12-12 20:18:25.485882: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 20:18:25.619662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/lib/native:/docker/usr/lib/hadoop/lib/native:/docker/usr/lib/hadoop-lzo/lib/native
2022-12-12 20:18:25.619692: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
202